In [1]:
import json
with open("qwen_asnwer.json", "r", encoding="utf-8") as file:
    dev_data = json.load(file)

In [9]:
import re
def extract_numbers(text):
    # 정규 표현식을 사용하여 대괄호 안에 있는 숫자를 찾음
    numbers = re.findall(r'\[(\d+)\]', text)
    return int(num) for num in numbers  # 숫자를 정수로 변환

In [11]:
for data in dev_data:
    answer, supporting_sentences = "", ""
    if "**Answer**:" in data["generated_text"] and "**Supporting Sentences**:" in data["generated_text"]:
        answer = data["generated_text"].split("**Answer**:")[1].split("**Supporting Sentences**:")[0].strip()
        supporting_sentences = data["generated_text"].split("**Answer**:")[1].split("**Supporting Sentences**:")[1].strip()
        supporting_sentences = extract_numbers(supporting_sentences)
    
    else:
        print("hi")
    if answer == "" or supporting_sentences == "":
        print("hi")
    print(answer, supporting_sentences)

No [4]
Secretary of State for Constitutional Affairs [21]
No [23]
Greenwich Village, New York City [10]
S.M. Entertainment [13]
Annie Morton [1]
No, Local H and For Against are not both from the United States. [21]
David Weissman [20]
1999 [29]
Ernest Mangnall [7]
America East Conference-North [15]
Yes [13]
Robert Erskine Childers [18]
Eduardo Troconis [21]
Jim Cummings [42]
Wolf Alice [1]
Peter Schmeichel [27]
Nancy Sinatra [7]
No [6]
Henry J. Kaiser [21]
The adventure in "Tunnels and Trolls" is called "Arena of Khazan". [31]
2000 [43]
Fujioka, Gunma [16]
François Coli [17]
Letters to Cleo [2]
Carlo Rovelli [19]
World War II [27]
No [13]
New York City [39]
Mumbai [25]
Firth of Forth [10]
No [14]
2009 Big 12 Conference football season [9]
32 [43]
Indianapolis Motor Speedway [15]
Franco Corelli [16]
Savan Kotecha, Ilya Salmanzadeh, Max Martin, and Ali Payami [14]
Oversteering technique [15]
Keith Bostic is younger than Jerry Glanville. [28]
4,505 [8]
No [14]
Steven Spielberg [8]
Emma Bu